<a href="https://colab.research.google.com/github/LakshmiSHR/AIML/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# sms_spam_tfidf_logreg.py
# TF-IDF + Logistic Regression spam detector.
# Prints precision/recall/F1 and top positive/negative features.
# Saves confusion matrix figure.

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Local and remote dataset paths
LOCAL_PATH = "SMSSpamCollection"  # fallback local filename (optional)
REMOTE_TSV = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv" # Updated URL


def load_data():
    """Load the SMS spam dataset (local or remote)."""
    if os.path.exists(LOCAL_PATH):
        df = pd.read_csv(LOCAL_PATH, sep="\t", header=None, names=["label", "message"])
        return df
    try:
        df = pd.read_csv(REMOTE_TSV, sep="\t", header=None, names=["label", "message"])
        return df
    except Exception as e:
        raise RuntimeError(
            "Could not load SMS dataset. Place 'SMSSpamCollection' next to this script "
            "or ensure internet is available."
        ) from e


def main():
    os.makedirs("outputs/sms", exist_ok=True)
    df = load_data()

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        df["message"].values,
        df["label"].values,
        test_size=0.2,
        random_state=42,
        stratify=df["label"].values
    )

    # TF-IDF vectorizer
    vect = TfidfVectorizer(
        lowercase=True,
        stop_words="english",
        ngram_range=(1, 2),
        max_df=0.95,
        min_df=2
    )
    Xtr = vect.fit_transform(X_train)
    Xte = vect.transform(X_test)

    # Logistic Regression model
    clf = LogisticRegression(penalty="l2", solver="liblinear", max_iter=1000)
    clf.fit(Xtr, y_train)
    y_pred = clf.predict(Xte)

    # Classification report
    print("\nClassification report (test):")
    print(classification_report(y_test, y_pred, zero_division=0))
    print(f"Test accuracy: {accuracy_score(y_test, y_pred):.4f}")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=["ham", "spam"])
    disp = ConfusionMatrixDisplay(cm, display_labels=["ham", "spam"])
    disp.plot(cmap="Blues")
    plt.title("SMS Spam Confusion Matrix")
    plt.tight_layout()
    plt.savefig("outputs/sms/cm_sms.png", dpi=200)
    plt.close()
    print("Saved: outputs/sms/cm_sms.png")

    # Top positive/negative (toward "spam")
    feature_names = np.array(vect.get_feature_names_out())

    # For binary classification, clf.coef_ has shape (1, n_features).
    # The coefficients clf.coef_[0] correspond to the class at clf.classes_[1].
    # We want coefficients for 'spam'.
    # Find the index of 'spam' in the classes array
    spam_class_index = np.where(clf.classes_ == "spam")[0][0]

    # Get the coefficients. They are always in clf.coef_[0] for binary classification.
    coefs_raw = clf.coef_[0]

    # If 'spam' is the first class (index 0), then clf.classes_[1] is 'ham'.
    # In this case, coefs_raw represents 'ham' coefficients, so negate for 'spam'.
    if spam_class_index == 0:
        coefs = -coefs_raw
    else: # 'spam' is the second class (index 1), so coefs_raw directly represents 'spam' coefficients.
        coefs = coefs_raw

    top_pos = np.argsort(coefs)[-20:][::-1]
    top_neg = np.argsort(coefs)[:20]

    print("\nTop 20 SPAM-indicative features:")
    for f, w in zip(feature_names[top_pos], coefs[top_pos]):
        print(f"{f:30s} {w: .3f}")

    print("\nTop 20 HAM-indicative features:")
    for f, w in zip(feature_names[top_neg], coefs[top_neg]):
        print(f"{f:30s} {w: .3f}")


if __name__ == "__main__":
    main()



Classification report (test):
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       966
        spam       1.00      0.77      0.87       149

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115

Test accuracy: 0.9695
Saved: outputs/sms/cm_sms.png

Top 20 SPAM-indicative features:
txt                             4.305
mobile                          3.542
uk                              3.470
claim                           3.445
www                             3.276
reply                           3.066
stop                            3.025
free                            2.956
service                         2.832
150p                            2.718
prize                           2.561
text                            2.485
com                             2.431
50                              2.381
new                     